# Generate PMTiles 

Take all the Parquet files, generate their FlatGeoBuff versions and generate regional PMTiles from those. Then merge all PMTiles to a single file for the final application.

In [1]:
import os
import tempfile

import geopandas as gpd
from tqdm.auto import tqdm

In [9]:
v = "v10"

In [10]:
pmtiles_destination = f"/data/uscuni-ulce/pmtiles_{v}/"

In [11]:
os.mkdir(pmtiles_destination)
os.mkdir(pmtiles_destination + "regions")

In [12]:
regions = gpd.read_parquet("/data/uscuni-ulce/regions/cadastre_regions_hull.parquet")

## Buildings

In [13]:
folder = f"/data/uscuni-ulce/processed_data/clusters/{v}/"

In [14]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}clusters_{region}_{v}.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}regions/{region}.pmtiles -l buildings -pf -pk -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [15]:
os.system(f"tile-join -o {pmtiles_destination}complete.pmtiles -f -pk {pmtiles_destination}regions/*.pmtiles")

0

## Morphotopes

In [13]:
folder = "/data/uscuni-ulce/processed_data/morphotope_clusters/"

In [14]:
# os.mkdir(pmtiles_destination + "morphotopes")

In [15]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}{region}_clusters.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}morphotopes/{region}.pmtiles -z 12 -l morphotopes --coalesce-densest-as-needed -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [16]:
os.system(f"tile-join -o {pmtiles_destination}morphotopes.pmtiles -f -pk {pmtiles_destination}morphotopes/*.pmtiles")

0

## 250m grid

In [17]:
folder = "/data/uscuni-ulce/grid_250/"

In [18]:
# os.mkdir(pmtiles_destination + "grid_250")

FileExistsError: [Errno 17] File exists: '/data/uscuni-ulce/pmtiles/grid_250'

In [ ]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}{region}_clusters.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}grid_250/{region}.pmtiles -z 11 -l grid --coalesce-densest-as-needed -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [ ]:
os.system(f"tile-join -o {pmtiles_destination}grid_250.pmtiles -f -pk {pmtiles_destination}grid_250/*.pmtiles")

## H3

In [21]:
folder = "/data/uscuni-ulce/h3/"

In [18]:
# os.mkdir(pmtiles_destination + "h3")

In [22]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}{region}_clusters.pq").to_crs(4326).to_file(file_path)
        os.system(f"""tippecanoe -o {pmtiles_destination}h3/{region}.pmtiles -z 12 -l h3 --coalesce-densest-as-needed -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/847 [00:00<?, ?it/s]

In [23]:
os.system(f"tile-join -o {pmtiles_destination}h3.pmtiles -f -pk {pmtiles_destination}h3/*.pmtiles")

0

In [24]:
os.system("aws s3 --profile uscuni cp /data/uscuni-ulce/pmtiles/h3.pmtiles s3://uscuni-public")

upload: ../../../../../data/uscuni-ulce/pmtiles/h3.pmtiles to s3://uscuni-public/h3.pmtiles


0